In [1]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
from tensorflow.keras.preprocessing import image
import os

In [ ]:
# Configuración mejorada
# Usar las variables ya definidas en el notebook
# img_size, batch_size, dataset, train_datagen, val_datagen ya están definidos

# Generadores de datos
train_generator = train_datagen.flow_from_directory(
    dataset + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = val_datagen.flow_from_directory(
    dataset + "/train",
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

NameError: name '__file__' is not defined

In [ ]:
# Modelo CNN mejorado con regularización
def create_improved_model(num_classes):
    model = Sequential([
        # Primer bloque convolucional
        Conv2D(32, (3,3), activation='relu', input_shape=(224,224,3)),
        BatchNormalization(),
        Conv2D(32, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Segundo bloque convolucional
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Tercer bloque convolucional
        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        Conv2D(128, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Cuarto bloque convolucional
        Conv2D(256, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(2,2),
        Dropout(0.25),
        
        # Clasificador
        Flatten(),
        Dense(512, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    return model

# Crear el modelo mejorado
improved_model = create_improved_model(train_generator.num_classes)

# Compilar con learning rate más bajo
improved_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks para evitar overfitting
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )
]

In [ ]:
# Entrenar el modelo mejorado
print("Entrenando modelo mejorado...")
history = improved_model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,  # Más épocas pero con early stopping
    callbacks=callbacks,
    verbose=1
)

# Función mejorada para predicción
def predict_emotion_improved(model, img_path, class_labels):
    """
    Predice la emoción de una imagen con confianza
    """
    if not os.path.exists(img_path):
        return f"Imagen no encontrada: {img_path}"
    
    # Cargar y preprocesar imagen
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    
    # Predicción
    predictions = model.predict(img_array)
    predicted_class_idx = np.argmax(predictions, axis=1)[0]
    confidence = np.max(predictions) * 100
    
    # Obtener nombre de la clase
    emotion = class_labels[predicted_class_idx]
    
    # Mostrar todas las probabilidades
    print(f"\n=== PREDICCIÓN DE EMOCIÓN ===")
    print(f"Emoción predicha: {emotion}")
    print(f"Confianza: {confidence:.2f}%")
    print(f"\nDistribución de probabilidades:")
    for i, prob in enumerate(predictions[0]):
        print(f"{class_labels[i]}: {prob*100:.2f}%")
    
    return emotion, confidence

# Ejemplo de uso (descomenta para probar)
"""
# Obtener etiquetas de clase
class_labels = list(train_generator.class_indices.keys())

# Probar con una imagen
img_path = r"C:\IA\CourseIA\Unidad4\Tarea1\pruebados.jpg"
emotion, confidence = predict_emotion_improved(improved_model, img_path, class_labels)
"""


In [ ]:
# Evaluar el modelo mejorado
def evaluate_model_performance(model, validation_generator):
    """
    Evalúa el rendimiento del modelo
    """
    print("\n=== EVALUACIÓN DEL MODELO ===")
    
    # Evaluación en conjunto de validación
    val_loss, val_accuracy = model.evaluate(validation_generator, verbose=0)
    print(f"Precisión en validación: {val_accuracy:.4f}")
    print(f"Pérdida en validación: {val_loss:.4f}")
    
    return val_loss, val_accuracy

# Guardar el modelo mejorado
def save_improved_model(model, filename="emotion_recognition_improved.h5"):
    """
    Guarda el modelo entrenado
    """
    model.save(filename)
    print(f"Modelo guardado como: {filename}")

# Función para visualizar el historial de entrenamiento
def plot_training_history(history):
    """
    Visualiza las métricas de entrenamiento
    """
    import matplotlib.pyplot as plt
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Precisión
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_ylabel('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.legend()
    
    # Pérdida
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_ylabel('Loss')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

print("Modelo mejorado creado exitosamente!")
print("Características principales:")
print("- Data augmentation para evitar overfitting")
print("- Arquitectura más profunda con regularización")
print("- Batch normalization y dropout")
print("- Early stopping y reducción de learning rate")
print("- Evaluación detallada del rendimiento")